In [1]:
import os
import json
import h5py
import numpy as np

import robomimic
import robomimic.utils.file_utils as FileUtils
import robomimic.utils.env_utils as EnvUtils
import robomimic.utils.obs_utils as ObsUtils
import imageio
import tqdm
from robomimic.utils.file_utils import create_hdf5_filter_key

ROBOMIMIC WARNING(
    No private macro file found!
    It is recommended to use a private macro file
    To setup, run: python /home/ns1254/mimicgen/envs/robomimic/robomimic/scripts/setup_macros.py
)


In [2]:
# dataset_path = "/home/ns1254/robomimic/datasets/square/mh/image_v141.hdf5"
# dataset_path = "/home/ns1254/robomimic/datasets/can/mh/image_v141.hdf5"
# dataset_path = "/home/ns1254/dataset_mimicgen/kitchen82_31.hdf5"
# dataset_path = "/home/ns1254/dataset_mimicgen/coffee_gfs_109_2.hdf5"

# dataset_path = "/home/ns1254/robomimic/datasets/can/ph/image_v141.hdf5"
# dataset_path = "/home/ns1254/robomimic/datasets/lift/ph/image_v141.hdf5"

dataset_path = "/home/ns1254/data_franka/franka_bellpepper_g80.hdf5"

f_org = h5py.File(dataset_path, "r")
demos = list(f_org["data"].keys())

In [3]:
lengths=[]
for demo_name in demos:
    demo=f_org['data'][demo_name]
    num_samples=demo.attrs['num_samples']
    lengths.append(num_samples)

lengths=np.array(lengths)

print('Number of demos: ', len(demos))
print('Max length: ', np.max(lengths))
print('Min length: ', np.min(lengths))
print('Mean length: ', np.mean(lengths))

Number of demos:  80
Max length:  237
Min length:  113
Mean length:  157.8625


In [4]:
f_org['mask'].keys()

<KeysViewHDF5 ['good']>

In [5]:
demos[:3]

['demo_0', 'demo_1', 'demo_10']

In [6]:
f_org['mask'].keys()

<KeysViewHDF5 ['good']>

In [7]:
demos=[demo.decode('utf-8') for demo in f_org['mask']['20_percent']]
demos1=[demo.decode('utf-8') for demo in f_org['mask']['20_percent_train']]
demos2=[demo.decode('utf-8') for demo in f_org['mask']['20_percent_valid']]
len(demos), len(demos1), len(demos2)

(40, 36, 4)

In [7]:
demos =[demo.decode('utf-8') for demo in f_org['mask']['good']]

In [8]:
# good_demos1=[demo.decode('utf-8') for demo in f_org['mask']['better_operator_1']]
# good_demos2=[demo.decode('utf-8') for demo in f_org['mask']['better_operator_2']]
# len(good_demos1), len(good_demos2)

In [8]:
demos=sorted(demos, key=lambda x: int(x.split('_')[-1]))

In [10]:
good_demos=demos
good_demos, len(good_demos)

(['demo_0',
  'demo_1',
  'demo_2',
  'demo_3',
  'demo_4',
  'demo_5',
  'demo_6',
  'demo_7',
  'demo_8',
  'demo_9',
  'demo_10',
  'demo_11',
  'demo_12',
  'demo_13',
  'demo_14',
  'demo_15',
  'demo_16',
  'demo_17',
  'demo_18',
  'demo_19',
  'demo_20',
  'demo_21',
  'demo_22',
  'demo_23',
  'demo_24',
  'demo_25',
  'demo_26',
  'demo_27',
  'demo_28',
  'demo_29',
  'demo_30',
  'demo_31',
  'demo_32',
  'demo_33',
  'demo_34',
  'demo_35',
  'demo_36',
  'demo_37',
  'demo_38',
  'demo_39',
  'demo_40',
  'demo_41',
  'demo_42',
  'demo_43',
  'demo_44',
  'demo_45',
  'demo_46',
  'demo_47',
  'demo_48',
  'demo_49',
  'demo_50',
  'demo_51',
  'demo_52',
  'demo_53',
  'demo_54',
  'demo_55',
  'demo_56',
  'demo_57',
  'demo_58',
  'demo_59',
  'demo_60',
  'demo_61',
  'demo_62',
  'demo_63',
  'demo_64',
  'demo_65',
  'demo_66',
  'demo_67',
  'demo_68',
  'demo_69',
  'demo_70',
  'demo_71',
  'demo_72',
  'demo_73',
  'demo_74',
  'demo_75',
  'demo_76',
  'demo_7

In [11]:
len(demos)

80

In [12]:
# f_org.close()

In [12]:
# demos2copy=[demo.decode('utf-8') for demo in f_org['mask']['20_percent']] 
demos2copy=good_demos 
# demos2copy=demos2
len(demos2copy)

80

In [13]:
demos2copy[:5]

['demo_0', 'demo_1', 'demo_2', 'demo_3', 'demo_4']

In [14]:
# fn='/home/ns/collect_robomimic_demos/PickPlaceCan_02_09_2024_04_37PM_sojib/videos/expert/expert2.txt'
# with open(fn, 'r') as f:
#     file_demos = f.readlines()
# demos = [x.strip().replace(".mp4","").strip() for x in file_demos]
# len(demos), demos[:5]

In [15]:
demo_name = "demo_1"
b_demo=f_org['data'][demo_name]
obss = b_demo['obs']
obss.keys()

<KeysViewHDF5 ['agentview_rgb', 'agentview_rgb_depth', 'ee_states', 'eye_in_hand_rgb', 'eye_in_hand_rgb_depth', 'gripper_states', 'joint_states']>

In [18]:
dps = obss['agentview_rgb_depth'][:].reshape(-1, 128, 128, 1)
dps.shape

(145, 128, 128, 1)

In [19]:
dataset_path_sub =dataset_path
dataset_path_sub=dataset_path_sub.replace(".hdf5", "")+"_sub.hdf5"
dataset_path_sub

'/home/ns1254/data_franka/franka_bellpepper_g80_sub.hdf5'

In [20]:
f_sub = h5py.File(dataset_path_sub, "w")

In [21]:
keys_to_copy=demos2copy
len(keys_to_copy)

80

In [22]:
f_sub.create_group("data")
f_sub.create_group("mask")

<HDF5 group "/mask" (0 members)>

In [22]:
# attrs=f_org["data"].attrs["env_args"]
# f_sub["data"].attrs["env_args"] = attrs

In [23]:
i=0
demos=[]
for demo_name in tqdm.tqdm(keys_to_copy):
    b_demo=f_org['data'][demo_name]
    num_samples=b_demo.attrs['num_samples']
    # model_file=b_demo.attrs['model_file']

    bdemo_name="demo_"+str(i)
    demos.append(bdemo_name)
    i = i+1

    f_sub["data"].create_group(bdemo_name)


    ks=['actions']
    for k in ks:
        kdata = b_demo[k]
        f_sub["data"][bdemo_name].create_dataset(k, data=kdata)

    obs_keys =b_demo['obs'].keys()
    # next_obs_keys=b_demo['next_obs'].keys()
    for k in obs_keys:
        if '_depth' in k:
            kdata = b_demo['obs'][k][:].reshape(-1, 128, 128, 1) 
        else:
            kdata = b_demo['obs'][k]
        f_sub["data"][bdemo_name].create_dataset(f'obs/{k}', data=kdata)
    # for k in next_obs_keys:
    #     kdata = b_demo['next_obs'][k]
    #     f_sub["data"][bdemo_name].create_dataset(f'next_obs/{k}', data=kdata)

    f_sub["data"][bdemo_name].attrs["num_samples"] = num_samples
    # f_sub["data"][bdemo_name].attrs["model_file"] = model_file
 
# f_sub.close()
# f_org.close()

100%|██████████| 80/80 [00:02<00:00, 37.38it/s]


In [22]:
# demos

In [24]:
# demos=demos2copy
demos= np.array(demos, dtype='S8')  
demos.shape, demos[:4]

((80,), array([b'demo_0', b'demo_1', b'demo_2', b'demo_3'], dtype='|S8'))

In [25]:
hdf5_path=dataset_path_sub 

filter_keys=sorted([elem for elem in demos])
filter_name="good"
filter_lengths = create_hdf5_filter_key(hdf5_path=hdf5_path, demo_keys=filter_keys, key_name=filter_name)

In [26]:
f_sub['mask'].keys()

<KeysViewHDF5 ['good']>

In [36]:
# [demo.decode('utf-8') for demo in f_sub['mask']['20_percent_train'] ] 

In [28]:
obss = f_sub['data'][demos[0]]['obs']
obss['agentview_rgb_depth'].shape

(156, 128, 128, 1)

In [29]:
f_sub.close()
f_org.close()

In [23]:
i

30

In [2]:
dataset_path = "/home/ns1254/robomimic/datasets/can/ph/can_ph_image_20p.hdf5"

In [3]:
f_org = h5py.File(dataset_path, "r+")
demos = list(f_org["data"].keys())

In [9]:
problem_info='{"problem_name": "robomimic_can_ph", "domain_name": "robosuite", "language_instruction": "pick up the coke and place it on the bottom right corner."}'

In [10]:
f_org["data"].attrs["problem_info"] = problem_info

In [11]:
f_org["data"].attrs.keys()

<KeysViewHDF5 ['env_args', 'problem_info']>

In [12]:
problem_info = json.loads(f_org["data"].attrs["problem_info"])
language_instruction = "".join(problem_info["language_instruction"])
language_instruction

'pick up the coke and place it on the bottom right corner.'

In [13]:
f_org.close()